<a href="https://colab.research.google.com/github/Yu-WEN-creative/ChatGPT-Web/blob/main/Denoiser_Example_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U denoiser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for denoiser: filename=denoiser-0.1.5-py3-none-any.whl size=43826 sha256=5ee5444637a2be00bd22537e1da8c57a8b449b2c54486a9e21cc59d355028eb0
  Stored in directory: /root/.cache/pip/wheels/f1/57/3f/7ce0ae586d457c598b2cb275a37801ff5a705f656ba7a0215e
  Created wheel for pystoi: filename=pystoi-0.3.3-py2.py3-none-any.whl size=7779 sha256=5dc9b19ec3283de8765eff0b3ab85b3c533167d09e7a325d6042d9dec5465348
  Stored in directory: /root/.cache/pip/wheels/3b/ca/9e/5b5d6e5e109322303b50d21918ad2bd7d50a2a0775c11e08e8
  Created wheel f

# New section

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#视频在google云盘中的路径
!cp -r /content/drive/MyDrive/in/test.MP4 ./

In [9]:
!mkdir out

mkdir: cannot create directory ‘out’: File exists


In [10]:
!mkdir in

mkdir: cannot create directory ‘in’: File exists


In [16]:
#分离音轨 并且转换成模型表现较好的采样率16000
!ffmpeg -i test.MP4 -acodec pcm_s16le -ar 16000 -vn ./in/tmpaud.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [17]:
#音频切片分割成60秒一段的小音频
!ffmpeg -i ./in/tmpaud.wav -f segment -segment_time 60 -c copy ./in/output%03d.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [18]:
!rm ./in/tmpaud.wav

In [19]:
#降噪
!python3 -m denoiser.enhance --noisy_dir=in/ --out_dir=out/  --master64 --device cuda

INFO:denoiser.pretrained:Loading pre-trained real time H=64 model trained on DNS and Valentini.
INFO:__main__:Generate enhanced files | 1/6 | 0.8 it/sec
INFO:__main__:Generate enhanced files | 2/6 | 0.9 it/sec
INFO:__main__:Generate enhanced files | 3/6 | 1.0 it/sec
INFO:__main__:Generate enhanced files | 4/6 | 1.0 it/sec
INFO:__main__:Generate enhanced files | 5/6 | 1.1 it/sec
INFO:__main__:Generate enhanced files | 6/6 | 1.2 it/sec


In [20]:
#删除不必要的噪音文件，如果是歌曲 人声分离的话可以保留
! rm ./out/*noisy.wav

In [21]:
#将路径写入txt
files = os.listdir("./out/")
txt = open("input.txt","w")
txt.truncate(0)
for i in range(len(files)):
  txt.write("file"+" "+"out/output{number}_enhanced.wav\n".format(number=str(i).zfill(3)))
txt.close()

In [22]:
#将切片的音频拼接回来
!ffmpeg -f concat -safe 0 -i input.txt -c copy output.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [24]:
#将音轨添加回视频
!ffmpeg -i test.MP4 -i output.wav -map 0:v -map 1:a -c:v copy -c:a aac -b:a 256k -shortest out/output.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [26]:
!cp out/output.mp4 /content/drive/MyDrive/in